In [3]:
%pip install -q opencv-python torch scikit-learn torchvision pillow pandas requests setuptools 
# %pip install dlib



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
import dlib
import cv2
import numpy as np

vid = cv2.VideoCapture(0)
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
rect_size = 1

# Constants for drowsiness detection
EAR_THRESHOLD = 0.30  # Eye aspect ratio threshold for detecting drowsiness
EAR_CONSEC_FRAMES = 30  # Number of consecutive frames for which EAR is below threshold to consider drowsiness

def eye_aspect_ratio(eye):
    # Compute the euclidean distances between the two sets of vertical eye landmarks (x, y)-coordinates
    A = np.linalg.norm(eye[1] - eye[5])
    B = np.linalg.norm(eye[2] - eye[4])
    # Compute the euclidean distance between the horizontal eye landmark (x, y)-coordinates
    C = np.linalg.norm(eye[0] - eye[3])
    # Compute the eye aspect ratio
    ear = (A + B) / (2.0 * C)
    return ear

def detect_drowsiness(ear, prev_ears):
    prev_ears.append(ear)
    if len(prev_ears) > EAR_CONSEC_FRAMES:
        prev_ears.pop(0)
    avg_ear = np.mean(prev_ears)
    if avg_ear < EAR_THRESHOLD:
        return True
    else:
        return False

prev_ears = []

while True:
    ret, frame = vid.read()
    if ret:
        image_gs = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = detector(image_gs)
        for face in faces:
            landmarks = predictor(image_gs, face)
            left_eye = []
            right_eye = []
            # Extracting left eye and right eye landmarks
            for n in range(36, 42):
                x = landmarks.part(n).x
                y = landmarks.part(n).y
                left_eye.append((x, y))
            for n in range(42, 48):
                x = landmarks.part(n).x
                y = landmarks.part(n).y
                right_eye.append((x, y))
            # Compute eye aspect ratios
            left_ear = eye_aspect_ratio(np.array(left_eye))
            right_ear = eye_aspect_ratio(np.array(right_eye))
            avg_ear = (left_ear + right_ear) / 2.0
            # Detect drowsiness
            drowsy = detect_drowsiness(avg_ear, prev_ears)
            if drowsy:
                cv2.putText(frame, "DROWSY", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
            else:
                cv2.putText(frame, "AWAKE", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        cv2.imshow("frame", frame)
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break

vid.release()
cv2.destroyAllWindows()

2024-05-06 22:04:53.956 Python[76363:15461201] WARNING: Secure coding is not enabled for restorable state! Enable secure coding by implementing NSApplicationDelegate.applicationSupportsSecureRestorableState: and returning YES.
